# Binning. Straighten your data


## Introduction & Motivation

Binning is the common name for the set of approaches we can use for feature possible values range reducing. For example, we can split the continuous feature range into several parts and treat it like the categorical one. On the other side, we can merge some classes of the categorical variable. But why?

First of all, the reasons depend on the type of feature we bin. For categorical ones, it's not good if we don't have enough observations in some classes. So our model won't be able to make confident predictions for this class (underfitting). Also binning is obviously effective mean against outliers in numerical features: every extremely huge or small characteristic value is hidden as another instance of the top or bottom bin.

It's especially important that binning allows us to find and process non-monotonicity in numerical variables and hence we can make linear models work better without resorting to unstable and uninterpretable polynomial features.

Another point of linear model improvement and the common advantage of the binning technique for both categorical and numerical features is missing values processing. Also, you can get a deeper understanding of your data and make feature selection using some special metrics for binning results evaluation.

The most popular application of binning is a bank scorecard building. So in the tutorial, we will talk about the task - binary classification (are customer good or bad?) and the model - logistic regression. But I'm sure that these concepts are applicable for linear regression tasks too.

So, how does it work?

## Short Algorithms Review

The bank scoring theoretician Lyn C. Thomas said that binning technique "is as much art as it is science". That's true and obviously, there are many algorithms of binning.

Trivial way is splitting into equal parts. We can do it cutting the range into intervals with equal width (same variable band into each bin) or with equal size (same number of observations).  But these methods don't count target distribution at all. Also, we don't have any criteria for the choice of optimal split-points.

The more advanced path is to apply powerful statistical tests. The main idea is quite simple too.

OK, we start with our previous step and for clarity let's split feature range into a number of the smallest intervals (one observation in each bin). As we mentioned in the introduction we are deciding a binary classification task, such as prediction of bank delinquencies. So we can write a target value 1 or 2 (for good and bad clients respectively) in every small bin on a training set. After adding the column of feature values we should arrange our table in feature ascending order. Imagine that we got something like this.

<img src="./1.png" />

Now for every pair of adjacent bins, we can build a contingency table. It presents a frequency distribution of instances among two binary variables: the bin in pair and the target value.

<img src="./2.png" />

Statistics is starting! We are looking for similar pairs of adjacent bins to merge them, isn't it? So our null hypothesis stating is that bins have the same one-two frequency distribution and observed difference arose by chance. Let's compute expected values of frequency in this case (don't scare they can be fractions!) and check for every pair how far it from being siblings. We can do it using Pearson's chi-squared distance.

<img src="./3.png" />

Here I have to note that it is not a real statistical test because there are some constraints for Pearson's chi-squared test (we need more observations in each cell). But since we don't compute the p-value it is not very crucial. Apart from that, it's better to initialize bins with much more instances in each one in real-life cases.

OK, we have calculated statistics for all the pairs. There is a zero division problem for when adjacent bins have equal target distribution but we skip it and consider Pearson's distance is zero in these cases.

<img src="./4.png" />

We merge the most similar bins (with minimal chi-squared) and recalculate statistics and repeat and repeat again until we have stepped some predefined threshold for the greatest chi-square distance. One could use p-value to set this threshold but it is important to remember about mentioned constraints.

<img src="./5.png" />

It is possible to apply another statistical test or other similarity measures. However, the main idea of optimal statistical binning is splitting the characteristic possible answer set into intervals which have the most different good-bad proportion.

The described approach is widespread but it is not only one. There are tree-based algorithms: conditional inference trees or traditional decision trees with information gain as splitting criteria.

If we have strong prior knowledge that the bad rate (or target rate, bad clients percentage) will be monotone in some variable then the best way is using Maximum likelihood monotone coarse classifier (MLMCC). It provides a strict linear dependence between predictor and bad rate i.e. the maximum likelihood splitting in full concordance with its name.

OK, I hope you feel that there is not any rocket science in binning. So feel free to be an artist. You can combine some of these methods to do your best for decision your own task. Links to the useful sources where you can get more detail are in the references at the end of the tutorial.

## WoE & IV

Weight of evidence (WoE) and Information Value (IV) are the most commonly used metrics for evaluation binning results.
WoE is calculated for each separate bin and shows how much does target distribution in the bin distinct from distribution on the whole train set:

<img src="./7.png" />
Here $\sum{G_i}$ and $\sum{B_i}$ are amounts of good and bad instances in the ith bin. Obviously, if we put random samples in the bin we would get nearly the same distribution in it and WoE would be zero. The WoE range is from -inf to +inf, so it presents how far is our bin from just a random sample.

To evaluate the predictive power of whole binned predictor one should use IV:

<img src="./6.png" />
By convention, we have some simple rules for predictor IV values: 
- less than 0.02: not useful predictor;
- 0.02 to 0.1: weak predictor;
- 0.1 to 0.3: medium predictor;
- 0.3 to 0.5: strong predictor;
- more than 0.5: suspicious predictor (should be checked for data leakage or some other mistakes).

So, we can evaluate, compare and select features using IV.

Another noteworthy trick is WoE-encoding of binned categorical predictors. It allows avoiding dataset explosion with a lot of dummy variables and it is similar to the target encoding technique. We just need to write in every bin its WoE value and launch logistic regression.

## Python Implementation

When I faced with binning at my job (of course, it's a bank) I was surprised that there wasn't suitable open source packages for it in Python. So I've coded it myself.

You can find the full code of my implementation of binning in file **pwlf_binner.py**.

The main idea of my algorithm is to approximate ROC-curve. We will lose the least amount if information about our data if we do so. You just should pass in constructor required number of bins.

<img src="./example.png" />

Under the hood, binner uses differential evolution minimisation algorithm.

Ok, let's check how it can help us.

## Let's check it

For model training, we will use data about bank customers and their delinquencies from [Yorko's Git](https://github.com/Yorko/mlcourse.ai).

We will n drop all NaN from dataset so that we will be able to compare logistic regression performance on binned and unbinned data.

In [0]:
# firstly you should update your hyperopt library 
# !pip install -U hyperopt

In [0]:
#prerequirements
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, atpe, anneal, rand, Trials
from pybinning import GiniBinner

In [0]:
data = pd.read_csv('./credit_scoring_train.csv')

In [4]:
data.head()

,client_id,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
0,0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0


In [0]:
target = data['Delinquent90']
X_train, X_val, y_train, y_val = train_test_split(
    data.drop(columns=['Delinquent90', 'client_id']), target,
    test_size=0.3, shuffle=True, random_state=19
)

Let's play with binner. Check docstring to find out all details about params.

In [6]:
%%time
binner = GiniBinner(
    max_bins = 9,
    min_size = 0.05,
    method = 'anneal',
    n_iter = 100,
    fast = False,
    random_state = 19,
    verbose = True
 )
X_train_binned = binner.fit_transform(X_train, y_train)

100%|██████████| 9/9 [00:07<00:00,  1.20it/s]

CPU times: user 7.48 s, sys: 37.9 ms, total: 7.52 s
Wall time: 7.54 s


You can find full information about binning results for each variable into report attribute. Check docstring to find out more about particular report parts.

In [7]:
binner.report['Income']

{'NaN bin': 0,
 'Result bins': [0, 1, 2, 3, 4, 5, 6, 7, 8],
 'bin_volume': {0: 10370,
  1: 3085,
  2: 6511,
  3: 7691,
  4: 507,
  5: 4494,
  6: 8291,
  7: 4942,
  8: 6609},
 'gini': 0.1511820231308756,
 'iv': 0.07322410682592491,
 'loss': 0.00176411595201957,
 'opt_thresholds': [1702.282157,
  3222.754931,
  4673.446655,
  4796.069725,
  5668.494176,
  7953.667579999999,
  10058.431922],
 'target_rate': {0: 0.05718418514946962,
  1: 0.08427876823338736,
  2: 0.09522346797726923,
  3: 0.082434013782343,
  4: 0.09467455621301775,
  5: 0.07320872274143302,
  6: 0.06018574357737305,
  7: 0.052812626467017405,
  8: 0.042063852322590405},
 'woe': {0: 0.1702513608846863,
  1: -0.24676019505824587,
  2: -0.3808806441407093,
  3: -0.22261585105965517,
  4: -0.37449300847060535,
  5: -0.09392840378768831,
  6: 0.11590454194759295,
  7: 0.25440442635411764,
  8: 0.49325015923022997}}

In [8]:
X_val_binned = binner.transform(X_val)

100%|██████████| 9/9 [00:00<00:00, 10.24it/s]


In [9]:
lr = LogisticRegression( )
lr.fit(X_train_binned, y_train)
y_pred = lr.predict_proba(X_val_binned)[:,1]
print('Gini is %.2f' % (100*(roc_auc_score(y_val, y_pred)*2 - 1)))

Gini is 62.66


Let's try fast version

In [10]:
%%time
binner_fst = GiniBinner(
    max_bins = 9,
    min_size = 0.05,
    method = 'anneal',
    n_iter = 100,
    fast = True,
    starts_from = 100,
    random_state = 19,
    verbose = True
 )
X_train_binned = binner_fst.fit_transform(X_train, y_train)

100%|██████████| 9/9 [00:05<00:00,  1.57it/s]

CPU times: user 5.72 s, sys: 98.2 ms, total: 5.82 s
Wall time: 5.76 s


In [11]:
X_val_binned = binner_fst.transform(X_val)

100%|██████████| 9/9 [00:00<00:00, 10.76it/s]


In [12]:
lr = LogisticRegression()
lr.fit(X_train_binned, y_train)
y_pred = lr.predict_proba(X_val_binned)[:,1]
print('Gini is %.2f' % (100*(roc_auc_score(y_val, y_pred)*2 - 1)))

Gini is 62.65


The same result a bit faster as we can see. But the difference is much more essential on large datasets both in time benefit and quality decreasing. So use this feature carefully.

Model without binning

In [13]:
data_full = data.dropna()
target_full = data_full['Delinquent90']
X_train_full, X_val_full, y_train_full, y_val_full = train_test_split(
    data_full.drop(columns=['Delinquent90', 'client_id']), target_full,
    test_size=0.3, shuffle=True, random_state=19
)
lr = LogisticRegression()
lr.fit(X_train_full, y_train_full)
y_pred_full = lr.predict_proba(X_val_full)[:,1]
print('Gini is %.2f' % (100*(roc_auc_score(y_val_full, y_pred_full)*2 - 1)))

Gini is 34.81


We have got great gain in our model quality!